# OpenAIAgent with Playgrounds Subgraph Connector

In this tutorial, we'll dive into how you can use the OpenAIAgent available through Llama_index in combination with the `PlaygroundsSubgraphConnector` to query data from any decentralized subgraph on the graph network via the Playgrounds API.


## Advantages of this tool:

- **Easy access to Decentralized Subgraphs (Datasets)**: No need for wallet or GRT management.
- **LLM x Blockchain data**: Develop Ai applications that leverage blockchain data seamlessly.
- **Intutive Analytics**: Query blockchain data using natural language.

## Need to know:

The OpenAIAgent allows developers to integrate a diverse range of tools and data sources with OpenAI's language models. When combined with the PlaygroundsSubgraphConnector, you can easily fetch and analyze data from decentralized subgraphs.

Prerequisites:

- An API key from OpenAI.
- An API key from Playgrounds: https://app.playgrounds.network/signup
- A subgraph identifier or deployment ID : https://thegraph.com/explorer

To learn more about Playgrounds, please visit our website : https://playgrounds.network/


In [30]:
import openai
from llama_index.agent import OpenAIAgent
from connector_agent_tool.base import PlaygroundsSubgraphConnectorToolSpec

def simple_query(
    user_prompt: str, 
    openai_api_key: str, 
    identifier: str, 
    playgrounds_api_key: str, 
    use_deployment_id: bool
):
    """
    Run a test querying the financialsDailySnapshots from Uniswap V3 subgraph using OpenAIAgent and Playgrounds API.
    
    Args:
        user_prompt (str): The query text to be used for the GraphQL request.
        openai_key (str): The API key for OpenAI.
        identifier (str): The identifier for the subgraph or deployment.
        playgrounds_api_key (str): The API key for Playgrounds.
        use_deployment_id (bool): Flag to use deployment id in the URL.
    """
    
    # Set the OpenAI API key
    openai.api_key = openai_api_key
    
    # Initialize the tool specification
    connector_spec = PlaygroundsSubgraphConnectorToolSpec(
        identifier=identifier, 
        api_key=playgrounds_api_key, 
        use_deployment_id=use_deployment_id
    )
    
    # Setup agent with the tool
    agent = OpenAIAgent.from_tools(connector_spec.to_tool_list())
    
    # Make a query using the agent
    response = agent.chat(user_prompt)
    return response


## Simple prompts

For users who are familiar with the structure and content of the subgraph they're working with, the process can be streamlined even further. Instead of crafting extensive GraphQL queries, you can simply describe the data you want in plain English.

In the example below, we're interested in the last 30 days of `financialsDailySnapshots` for specific fields: `timestamp`, `totalValueLockedUSD`, `dailyBorrowUSD`, and `dailyDepositUSD`

By specifying the entities and fields you're interested in, and providing any additional instructions for the agent, you can retrieve the desired data with minimal effort. This approach is especially useful for quick data retrieval tasks or when exploring a familiar subgraph.

Note: Always ensure your prompts are clear and specific to get the most accurate results from the agent.

In [31]:
openai_api_key = "YOUR_OPENAI_API_KEY"
identifier = "YOUR_SUBGRAPH_OR_DEPLOYMENT_IDENTIFIER"
playgrounds_api_key = "YOUR_PLAYGROUNDS_API_KEY"
use_deployment_id = False

In [29]:
user_prompt = "Retrieve financialsDailySnapshots from the past 30 days with fields timestamp, totalValueLockedUSD, dailyBorrowUSD, and dailyDepositUSD and order the results by timestamp in descending order"

print(simple_query(user_prompt,openai_api_key,identifier,playgrounds_api_key,use_deployment_id))

Here are the financialsDailySnapshots from the past 30 days, ordered by timestamp in descending order:

1. Timestamp: 1696873931
   - Total Value Locked (USD): 3126267433.658772049415380129309006
   - Daily Borrow (USD): 30763652.54292852483114525101874036
   - Daily Deposit (USD): 923201458.5187751911431148870272052

2. Timestamp: 1696808447
   - Total Value Locked (USD): 3229951271.816925155415242253881288
   - Daily Borrow (USD): 17830012.77776809051763455212308771
   - Daily Deposit (USD): 409129145.1275299793935969693446145

3. Timestamp: 1696723019
   - Total Value Locked (USD): 3224290884.340570418566263846330619
   - Daily Borrow (USD): 8850736.221998095010626573712151081
   - Daily Deposit (USD): 121494674.1717561814698093226803111

4. Timestamp: 1696636391
   - Total Value Locked (USD): 3244107008.248023771273621360007208
   - Daily Borrow (USD): 15372198.5539701966160708375346591
   - Daily Deposit (USD): 69069661.53502122638418541861423538

5. Timestamp: 1696549331
   - Tot

## Structured Prompts

While simple prompts are effective for quick queries, there might be situations where you need more structure and precision in your queries. Especially if you are not entirely certain about the content or when you want to ensure specific details are captured. In these cases, copying the GraphQL query from tools like the subgraph explorer becomes extremely handy.

In the example below, we've taken a direct GraphQL query from the subgraph explorer and provided it as a prompt, asking for a concise summary of the queried data.

This structured approach ensures that the OpenAIAgent gets all the necessary details, leading to more comprehensive and accurate results. It's especially beneficial when you are exploring unfamiliar subgraphs or want to ensure that specific fields and entities are included in the query.

Note: When copying GraphQL queries, always make sure the syntax is correct and the query is compatible with the target subgraph to avoid errors.

In [27]:
user_prompt = """
      {
        financialsDailySnapshots(orderBy: timestamp, orderDirection: desc, first: 5) {
          id
          timestamp
          totalValueLockedUSD
          totalBorrowBalanceUSD
          totalDepositBalanceUSD
        }
      }
      
    Query and provide a concise summary of the data
    """

print(simple_query(user_prompt, openai_api_key, identifier, playgrounds_api_key, use_deployment_id))


The data retrieved from the query includes the following financial information for the five most recent daily snapshots:

1. Snapshot ID: 19639
   - Timestamp: 1696871303
   - Total Value Locked (USD): 3,126,208,351.22
   - Total Borrow Balance (USD): 1,059,947,661.03
   - Total Deposit Balance (USD): 3,126,208,351.22

2. Snapshot ID: 19638
   - Timestamp: 1696808447
   - Total Value Locked (USD): 3,229,951,271.82
   - Total Borrow Balance (USD): 1,082,411,706.12
   - Total Deposit Balance (USD): 3,229,951,271.82

3. Snapshot ID: 19637
   - Timestamp: 1696723019
   - Total Value Locked (USD): 3,224,290,884.34
   - Total Borrow Balance (USD): 1,080,087,862.42
   - Total Deposit Balance (USD): 3,224,290,884.34

4. Snapshot ID: 19636
   - Timestamp: 1696636391
   - Total Value Locked (USD): 3,244,107,008.25
   - Total Borrow Balance (USD): 1,086,410,683.18
   - Total Deposit Balance (USD): 3,244,107,008.25

5. Snapshot ID: 19635
   - Timestamp: 1696549331
   - Total Value Locked (USD): 3,

## Using with Langchain

The `PlaygroundsSubgraphConnectorToolSpec` can be used with Langchain agents, by converting to a LangChain tool

In this example, we do the following:

- Create structured tools from the `PlaygroundsSubgraphInspectorToolSpec()`.
- Transform the connector tool into a format compatible with LangChain using `to_langchain_tool()`.
- Initialize the LangChain agent with the structured tool, in this case we use `StructuredTool.from_function()`
- Initialize the LangChain agent with `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` for chat output 

In [32]:
import os
import openai
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from connector_agent_tool.base import PlaygroundsSubgraphConnectorToolSpec
from typing import Union

def fetch_from_playgrounds(query: str) -> Union[dict, str]:
    """
    Fetch data from subgraph via Playgrounds API using the provided GraphQL query.
    
    Args:
        query (str): The GraphQL query string to execute.

    Returns:
        dict: The response from the GraphQL server if successful.
        str: Error message if the request fails.
    """
    # Instantiate the connector
    connector = PlaygroundsSubgraphConnectorToolSpec(
        identifier=identifier, 
        api_key=playgrounds_api_key, 
        use_deployment_id=use_deployment_id
    )
    # Use the graphql_request method to fetch data
    response = connector.graphql_request(query)
    return response

# Convert the fetch_from_playgrounds function into a StructuredTool is then used with LangChain
structured_tool = StructuredTool.from_function(fetch_from_playgrounds)

def simple_query_with_langchain(
    user_prompt: str, 
    openai_api_key: str, 
    identifier: str, 
    playgrounds_api_key: str, 
    use_deployment_id: bool
):
    """
    Querying subgraph using LangChain Agent and Playgrounds API.
    """
    
    # Set the OpenAI API key and Initialize the LLM
    openai.api_key = openai_key
    llm = OpenAI(temperature=0)
    
    # Initialize the agent with the previously created structured_tool
    agent_executor = initialize_agent(
        [structured_tool],
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    # Process the query using the LangChain agent
    response = agent_executor.run(user_prompt)
    return response


In [ ]:
openai_api_key = 'YOUR_OPENAI_API_KEY'
identifier = "YOUR_SUBGRAPH_OR_DEPLOYMENT_IDENTIFIER"
playgrounds_api_key = "YOUR_PLAYGROUNDS_API_KEY"
use_deployment_id = False

In [10]:
user_prompt = 'Query the first 5 days of financialsDailySnapshots for id, timestamp, totalValueLockedUSD, and dailyVolumeUSD.'

print(simple_query_with_langchain(user_prompt, openai_api_key, identifier, playgrounds_api_key, use_deployment_id))



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "fetch_from_playgrounds",
  "action_input": "query { financialsDailySnapshots(first: 5) { id timestamp totalValueLockedUSD dailyVolumeUSD } }"
}
```

Observation: {'data': {'financialsDailySnapshots': [{'id': '18751', 'timestamp': '1620171619', 'totalValueLockedUSD': '101976.3680947069357315540920392817', 'dailyVolumeUSD': '0'}, {'id': '18752', 'timestamp': '1620259158', 'totalValueLockedUSD': '128930008.0119693564848598071496047', 'dailyVolumeUSD': '14589148.89160808212877907657332154'}, {'id': '18753', 'timestamp': '1620345584', 'totalValueLockedUSD': '341711935.66063250446414077793037', 'dailyVolumeUSD': '202165514.672696141748426137929786'}, {'id': '18754', 'timestamp': '1620431997', 'totalValueLockedUSD': '469265156.9558929226412297565582368', 'dailyVolumeUSD': '345756118.0718718857741980736181714'}, {'id': '18755', 'timestamp': '1620518392', 'totalValueLockedUSD': '532058061.0991127647664666899727599', 'dailyVolume

# OpenAIAgent with Playgrounds Subgraph Inspector

The `PlaygroundsSubgraphInspectorToolSpec` is a tool designed for LLM agnets to seamlessly interface, query, introspect and analyze the schema of any subgraphs deployed on The Graph's decentralized network.

Queries are facilitated via the [Playgrounds API](https://playgrounds.network/).

Playgrounds API is a service provided by [Playgrounds Analytics](https://playgrounds.network) to streamline interfacing with decentralized subgraphs (indexed blockchain datasets).

## Advantages of this tool:

- **Seamless Integration with Decentralized Subgraphs**: Access decentralized datasets without the complexities of wallet or GRT management.
- **LLM x Blockchain data**: Develop Ai applications that leverage blockchain data seamlessly.
- **Rich Context**: Introspection provides the LLM agents with rich contextual understanding on subgraphs being queried. This in turn enhances the agents ability to generate insights on the underlying dataset indexed by the subgraph.

## How it works:

The OpenAIAgent, when used with the `PlaygroundsSubgraphInspectorToolSpec`, offers a robust method to fetch and decipher data from decentralized subgraphs.

Prerequisites:

- An API key from OpenAI.
- An API key from Playgrounds: https://app.playgrounds.network/signup
- A subgraph identifier or deployment ID : https://thegraph.com/explorer

To learn more about Playgrounds, please visit our website : https://playgrounds.network/

In [23]:
import openai
from llama_index.agent import OpenAIAgent
from introspector_agent_tool.base import PlaygroundsSubgraphInspectorToolSpec

def inspect_subgraph(
    openai_api_key: str,
    playgrounds_api_key: str,
    identifier: str,
    use_deployment_id: bool,
    user_prompt: str
):
    """
    Introspect a subgraph using OpenAIAgent and Playgrounds API with the provided parameters.
    
    Args:
        openai_api_key (str): API key for OpenAI.
        playgrounds_api_key (str): API key for Playgrounds.
        identifier (str): Identifier for the subgraph or deployment.
        use_deployment_id (bool): If True, uses deployment ID in the URL.
        user_prompt (str): User's question or prompt for the agent.
        
    Returns:
        str: Agent's response.
    """
    # Set the OpenAI API key
    openai.api_key = openai_api_key
    
    # Initialize the introspector with the provided parameters
    inspector_spec = PlaygroundsSubgraphInspectorToolSpec(
        identifier=identifier, 
        api_key=playgrounds_api_key, 
        use_deployment_id=use_deployment_id
    )
    
    # Integrate the tool with the agent
    agent = OpenAIAgent.from_tools(inspector_spec.to_tool_list())
    
    # Send the user prompt to the agent
    response = agent.chat(user_prompt)
    return response


### Example #1 : Summarize entire subgraph

In [ ]:
openai_api_key = "YOUR_OPENAI_API_KEY"
identifier = "YOUR_SUBGRAPH_OR_DEPLOYMENT_IDENTIFIER"
playgrounds_api_key = "YOUR_PLAYGROUNDS_API_KEY"
use_deployment_id = False

In [33]:
user_prompt="which fields will help me understand swap activities"
print(inspect_subgraph(user_prompt,openai_api_key,identifier,playgrounds_api_key,use_deployment_id))

To understand swap activities, you can consider the following fields:

1. Entity: Swap
   - logIndex: The index of the swap event in the log.
   - blockNumber: The block number in which the swap occurred.
   - tokenIn: The token that was swapped.
   - amountIn: The amount of tokenIn that was swapped.
   - amountInUSD: The equivalent value of amountIn in USD.
   - tokenOut: The token received in the swap.
   - amountOut: The amount of tokenOut received.
   - amountOutUSD: The equivalent value of amountOut in USD.

These fields provide information about the tokens involved in the swap, the amounts swapped, and their equivalent values in USD.


### Example #2: Insights on subgraph fields

In [27]:
user_prompt='which fields will help me understand the usage of the protocol'
print(inspect_subgraph(user_prompt,openai_api_key,identifier,playgrounds_api_key,use_deployment_id))

To understand the usage of the protocol, you can use the following fields:

1. Entity: DexAmmProtocol
   - schemaVersion
   - subgraphVersion
   - methodologyVersion
   - totalValueLockedUSD
   - protocolControlledValueUSD
   - cumulativeVolumeUSD
   - cumulativeSupplySideRevenueUSD
   - cumulativeProtocolSideRevenueUSD
   - cumulativeTotalRevenueUSD
   - cumulativeUniqueUsers
   - totalPoolCount
   - dailyUsageMetrics
   - hourlyUsageMetrics
   - financialMetrics

2. Entity: Protocol
   - schemaVersion
   - subgraphVersion
   - methodologyVersion
   - totalValueLockedUSD
   - protocolControlledValueUSD
   - cumulativeSupplySideRevenueUSD
   - cumulativeProtocolSideRevenueUSD
   - cumulativeTotalRevenueUSD
   - cumulativeUniqueUsers
   - totalPoolCount
   - dailyUsageMetrics
   - hourlyUsageMetrics
   - financialMetrics

3. Entity: LiquidityPool
   - inputTokens
   - outputToken
   - rewardTokens
   - isSingleSided
   - createdTimestamp
   - createdBlockNumber
   - totalValueLockedUSD


## Using with LangChain

Compared to the previous example of using with langchain, let's take a different approach here. The function `inspect_subgraph_with_langchain` provides a method to introspect subgraphs using LangChain and the Playgrounds API.

In this example, we do the following:
- Create an instance of `PlaygroundsSubgraphInspectorToolSpec`
- Transform the introspector tool into a format compatible with LangChain using `to_langchain_tool()`.
- Set up the Language Model with OpenAI(temperature=0).
- Initialize the LangChain agent with the tool and `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` for chat output 
  

In [11]:
import openai
from langchain.llms import OpenAI
from llama_index.agent import OpenAIAgent
from langchain.agents import initialize_agent, AgentType
from introspector_agent_tool.base import PlaygroundsSubgraphInspectorToolSpec

def introspect_and_summarize_subgraph() -> Union[dict, str]:
    """
    Introspects the subgraph and summarizes its schema into textual categories.
    """
    
    # Initialize the inspector
    inspector = PlaygroundsSubgraphInspectorToolSpec(
        identifier=identifier, 
        api_key=playgrounds_api_key, 
        use_deployment_id=use_deployment_id
    )
    # Use the introspect_and_summarize_subgraph method
    response = inspector.introspect_and_summarize_subgraph()
    return response

# Convert the introspect_and_summarize_subgraph function into a StructuredTool is then used with LangChain
structured_inspector_tool = StructuredTool.from_function(introspect_and_summarize_subgraph)

def inspect_subgraph_with_langchain(
    user_prompt: str, 
    openai_api_key: str, 
    identifier: str, 
    playgrounds_api_key: str, 
    use_deployment_id: bool
):
    """
    Introspects the subgraph using LangChain Agent and Playgrounds API.
    """
    
    # Set the OpenAI API key and Initialize the LLM
    openai.api_key = openai_api_key
    llm = OpenAI(temperature=0)
    
    # Initialize the agent with the previously created structured_inspector_tool
    agent_executor = initialize_agent(
        [structured_inspector_tool],
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    # Process the query using the LangChain agent
    response = agent_executor.run(user_prompt)
    return response


In [ ]:
openai_api_key = "YOUR_OPENAI_API_KEY"
identifier = "YOUR_SUBGRAPH_OR_DEPLOYMENT_IDENTIFIER"
playgrounds_api_key = "YOUR_PLAYGROUNDS_API_KEY"
use_deployment_id = False

In [12]:
user_prompt = "what are the fields in the financialsDailySnapshots?"
print(inspect_subgraph_with_langchain(user_prompt, openai_api_key, identifier, playgrounds_api_key, use_deployment_id))



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "introspect_and_summarize_subgraph",
  "action_input": {}
}
```


Observation: Category: Specific Entity Queries (Requires Arguments)
Description: These queries target a singular entity and require specific arguments (like an ID) to fetch data.
Generic Example:

            {
                entityName(id: "specific_id") {
                    fieldName1
                    fieldName2
                    ...
                }
            }
            

Detailed Breakdown:
  Entity: Deposit
    - logIndex
    - blockNumber
    - inputTokens
    - outputToken
    - inputTokenAmounts
    - outputTokenAmount
    - amountUSD

  Entity: DexAmmProtocol
    - schemaVersion
    - subgraphVersion
    - methodologyVersion
    - totalValueLockedUSD
    - protocolControlledValueUSD
    - cumulativeVolumeUSD
    - cumulativeSupplySideRevenueUSD
    - cumulativeProtocolSideRevenueUSD
    - cumulativeTotalRevenueUSD
    - cumulativeUniq